In [1]:
import numpy as np
from utils import UtilsRender
from game_entities import CapFactory, Arrow

# %%
from importlib import reload
import utils
import collision_resolver
utils = reload(utils)
ur = utils.UtilsRender()
collision_resolver = reload(collision_resolver)
CollisionResolver = collision_resolver.CollisionResolver

# Initial setup

In [2]:
w = 1920
h = 1080
radii = 25
X = np.array(
    [[  1275,  530],
    [1307,  470],
    [1310,  700],
    [1390,  560],
    [100,  100],
    ], dtype=np.float32
)
R = np.array([radii, radii, radii, radii, radii], dtype=np.float32)
M = np.array([1, 1, 1, 1, 1], dtype=np.float32)


In [3]:
# cap_factory = CapFactory(X=X)
# arrow = Arrow(
#     caps=cap_factory,
#     cap_index=0,
#     arrow_length=100,
#     angle=0
# )

In [4]:
ur.render_snapshot(X, R)

# Collision Resolver

In [5]:
V = np.array(
    [[4, 4],
    [2, -2],
    [-1.5, -1.5],
    [3, 3],
    [-3, 0],
    ], dtype=np.float32
)
cr = CollisionResolver(R=R, M=M, boundaries=(w, h))
# ur.render_snapshot(X_next, R)

In [6]:
def get_cap_velocity_modulus(V):
        """
        Get the velocity modulus of the cap
        Returns a matrix of shape (n, 1)
        """
        return np.linalg.norm(V, axis=1)

def is_system_moving(V):
    """
    Check if norm of V matrix is >= min_velocity
    """
    min_velocity = 0.1
    caps_mod_velocity = get_cap_velocity_modulus(V)
    return np.all(caps_mod_velocity < min_velocity)

In [7]:
V

array([[ 4. ,  4. ],
       [ 2. , -2. ],
       [-1.5, -1.5],
       [ 3. ,  3. ],
       [-3. ,  0. ]], dtype=float32)

In [13]:
%%time
positions = [X]
X_i = X.copy()

for i in range(5000):
    V_next = cr.resolve_collision(X=X_i, V=V)
    X_next = X_i + V_next
    positions.append(X_next)
    X_i = X_next
    V = V_next
    if is_system_moving(V_next):
        print(f"System stopped at iteration {i}")
        break

CPU times: user 128 ms, sys: 0 ns, total: 128 ms
Wall time: 128 ms


In [14]:
ur.render_motion(positions=positions, radius=R, add_delay=20)

In [22]:
ur.render_snapshot(positions[-1], R)

In [25]:
V_next = cr.resolve_collision(X=X_i, V=V)

In [24]:
V_next

array([[ 2.849689 ,  3.222341 ],
       [ 2.       , -2.       ],
       [-1.5      , -1.5      ],
       [ 4.1503134,  3.7776604],
       [ 3.       ,  0.       ]], dtype=float32)

# [DEBUG] Collision Resolver

In [233]:
X_next = X + V

## Edge Collider

In [234]:

edge_collisions = cr.get_edge_collisions(X_next=X_next)
edge_collisions

[(4, 0)]

## Cap Collider

In [235]:
cap_collisions = cr.get_cap_collisions(X_next=X_next)
cap_collisions

[(0, 3)]

In [236]:
cap_collisions

[(0, 3)]

## Mask collisions

In [237]:
r_edge_collisions = cr.resolve_collision_priority(edge_collisions=edge_collisions, cap_collisions=cap_collisions)

## Velocity of cap collisions

In [238]:
V_coll_cap_after, idx_coll_cap_final = cr.resolve_cap_collisions(cap_collisions=cap_collisions, V=V, X=X)

In [239]:
# Energy conservation and momentum conservation check
V_check_before = np.take(V, idx_coll_cap_final, axis=0)
M_check_before = np.take(M, idx_coll_cap_final, axis=0)
V_check_after = V_coll_cap_after
print(cr.check_momentum_conservation(V_initial=V_check_before, V_final=V_check_after, M=M_check_before))
print(cr.check_kinetic_energy_conservation(V_initial=V_check_before, V_final=V_check_after, M=M_check_before))

(array([2., 2.], dtype=float32), array([2., 2.], dtype=float32))
(4.0, 4.0000005)


## Velocity of edge collisions

In [240]:
# Take the first element of each of the tuples in the list r_edge_collisions
idx_coll_edge_final = [x[0] for x in r_edge_collisions]
idx_edge_index = [x[1] for x in r_edge_collisions]
V_coll_edge = np.take(V, idx_coll_edge_final, axis=0)

In [241]:
V_coll_edge_after = cr.resolve_edge_collisions(V=V_coll_edge, edge_indices=idx_edge_index)

In [242]:
V_coll_edge_after

array([[1., 0.]], dtype=float32)

## Bring all velocities back

In [243]:
Vf = V.copy()

In [244]:
# Iterate over the cap indices for the edge collisions
for i, idx in enumerate(idx_coll_edge_final):
    Vf[idx] = V_coll_edge_after[i]

# Iterate over the cap indices for the cap collisions
for i, idx in enumerate(idx_coll_cap_final):
    Vf[idx] = V_coll_cap_after[i]

In [245]:
Vf

array([[-0.14117643,  0.16470583],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 2.1411765 ,  1.8352942 ],
       [ 1.        ,  0.        ]], dtype=float32)

In [246]:
V

array([[ 2.,  2.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [-1.,  0.]], dtype=float32)